In [1]:
###List of importing
#For function "make_soup", "crawl_googleSearch"
import requests
from bs4 import BeautifulSoup

#For function "rawdata", Data_write_csv", Data_append_csv"
import csv

#For function "rawdata"
from collections import defaultdict

#For function "randomSleep"
import time, random
from random import *

In [2]:
def rawdata():
    columns = defaultdict(list) # each value in each column is appended to a list

    with open('DesignerData.csv',encoding="latin-1") as f:
        reader = csv.DictReader(f) # read rows into a dictionary format
        for row in reader: # read a row as {column1: value1, column2: value2,...}
            for (k,v) in row.items(): # go over each column name and value 
                columns[k].append(v) # append the value into the appropriate list
                                     # based on column name k

    return(columns['Product Name'],columns['Company Name'],columns['Designer'])

#OR using Pandas, below is just for reference, may have some typo
#import pandas as pd
#data = pd.read_csv("sdfds.csv")
#uname = data['uname'] #pandas.series
#uname = pd.to_list(uname)

In [3]:
def make_soup(url):

    headers = requests.utils.default_headers()
    headers.update({'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',})
    page = requests.get(url)

    return BeautifulSoup(page.text, 'html.parser')

In [4]:
def randomSleep():
        #sleeptime =  random.randint(60,180)
        sleeptime = uniform(5,10)
        time.sleep(sleeptime)
        print("time slept for " + str(sleeptime) + " seconds")

In [5]:
def crawl_info(ProductName,CompanyName,DesignerName): 
    try:
        data=[]
        
        ###format for google search
        ProductName = ProductName.replace('-','+') 
        ProductName = ProductName.replace(' ','+')
        
        CompanyName = CompanyName.replace(' ','+')
        DesignerName = DesignerName.replace(' ','+')
        
        GoogleSearch = 'https://www.google.com/search?q="{}"&hl={}'.format(ProductName,'en')
        
        ###1. Google Search by product name
        #Collect designer page of the product
        soup=make_soup(GoogleSearch)
        
        #Try avoiding to be blocked, with a random short wait
        randomSleep()
        
        #For testing
        #print(soup) 
        #print(soup.status_code) 
        #print(soup.prettify()) #show all coding in the web page
        
        
        PN_result = soup.find('div',id="resultStats")
        if PN_result:
            if PN_result.get_text():
                PN=PN_result.get_text().split()[-2]
            else:
                PN=0 #if did not match any documents
        else:
            PN=0 #if did not match any documents
        
        ###2. Google Search by company name    
        GoogleSearch = 'https://www.google.com/search?q="{}"&hl={}'.format(CompanyName,'en')
        
        soup=make_soup(GoogleSearch)
        
        randomSleep()
        
        CN_result = soup.find('div',id="resultStats")
        if CN_result:
            if CN_result.get_text():
                CN=CN_result.get_text().split()[-2]
            else:
                CN=0 #if did not match any documents
        else:
            CN=0 
        
        ###3. Google Search by designer name
        GoogleSearch = 'https://www.google.com/search?q="{}"&hl={}'.format(DesignerName,'en')
        
        soup=make_soup(GoogleSearch)
        
        randomSleep()
        
        DN_result = soup.find('div',id="resultStats")
        if DN_result:
            if DN_result.get_text():
                DN=DN_result.get_text().split()[-2]
            else:
                DN=0 #if did not match any documents
        else:
            DN=0   
        
        ###4. Combination: product name + company name
        GoogleSearch = 'https://www.google.com/search?q="{}"+OR+"{}"&hl={}'.format(ProductName,CompanyName,'en')
        
        soup=make_soup(GoogleSearch)
        
        randomSleep()
        
        Combine_result = soup.find('div',id="resultStats")
        if Combine_result:
            if Combine_result.get_text():
                Combine=Combine_result.get_text().split()[-2]
            else:
                Combine=0 #if did not match any documents
        else:
            Combine=0        
        
        ###5. Search product by location
        #hong kong
        GoogleSearch = 'https://www.google.com/search?q="{}"+location%3Ahong+kong&hl={}'.format(ProductName,'en')

        soup=make_soup(GoogleSearch)
        
        randomSleep()
        
        HK_result = soup.find('div',id="resultStats")
        if HK_result:
            if HK_result.get_text():
                HK=HK_result.get_text().split()[-2]
            else:
                HK=0 #if did not match any documents
        else:
            HK=0        
        
        #China
        GoogleSearch = 'https://www.google.com/search?q="{}"+location%3Achina&hl={}'.format(ProductName,'en')

        soup=make_soup(GoogleSearch)
        
        randomSleep()
        
        China_result = soup.find('div',id="resultStats")
        if China_result:
            if China_result.get_text():
                China=China_result.get_text().split()[-2]
            else:
                China=0 #if did not match any documents
        else:
            China=0
        
        ###6. Search product for a domain
        #Amazon.com
        GoogleSearch = 'https://www.google.com/search?q="{}"+site%3Aamazon.com&hl={}'.format(ProductName,'en')
        
        soup=make_soup(GoogleSearch)
        
        randomSleep()
        
        Amazon_result = soup.find('div',id="resultStats")
        if Amazon_result:
            if Amazon_result.get_text(): 
                Amazon=Amazon_result.get_text().split()[-2]
            else:
                Amazon=0 #if did not match any documents
        else:
            Amazon=0
        
        ProductName = ProductName.replace('+',' ')
        data.append(ProductName)
        data.append(PN)
        
        CompanyName = CompanyName.replace('+',' ')
        data.append(CompanyName)
        data.append(CN)  
        
        DesignerName = DesignerName.replace('+',' ')
        data.append(DesignerName)
        data.append(DN)
        
        data.append(Combine) 
        data.append(HK)    
        data.append(China) 
        data.append(Amazon)  
        
        Data_append_csv(data)

        ###7. Find similar content (Use URL)
        
    except Exception as e:
        print(e)

In [6]:
def Data_write_csv():
    with open("GoogleSearchResult.csv","w") as csv_file:
        writer = csv.writer(csv_file,delimiter=',')
        writer.writerow(["Product Name(PN)","Search Result by PN","Company Name(CN)",\
                         "Search Result by CN","Designer (D)","Search Result by D",\
                         "Search Result by PN+CN","Search Result by PN only in HK",\
                         "Search Result by PN only in China","Search Result by PN only in Amazon.com" ])

In [7]:
def Data_append_csv(data):
    with open("GoogleSearchResult.csv","a") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(data)

In [ ]:
def main():
    
    #Collect list of product name, company name & designer name from "DesignerData.csv"
    ListOfProductName = rawdata()[0] 
    ListOfCompanyName = rawdata()[1]
    ListOfDesignerName = rawdata()[2]
    
    #Create a new csv file called "GoogleSearchResult.csv" storing result of data crawling
    Data_write_csv() 
    
    x=0
    
    #1. Crawl comment from website & 2. put them into "GoogleSearchResult.csv" file
    for ProductName in ListOfProductName:
        
        CompanyName = ListOfCompanyName[x]
        DesignerName = ListOfDesignerName[x]
        
        crawl_info(ProductName,CompanyName,DesignerName)

        x +=1 #change to next product
main()

time slept for 6.93712889781964 seconds
time slept for 6.88333360495189 seconds
time slept for 6.082651940651997 seconds
time slept for 5.211991558676109 seconds
time slept for 9.444604567330483 seconds
time slept for 8.437468140325509 seconds
time slept for 9.007183066989402 seconds
time slept for 8.886408665277585 seconds
time slept for 8.309218167535255 seconds
time slept for 6.405479071421388 seconds
time slept for 6.905094808278725 seconds
time slept for 7.836807883506658 seconds
time slept for 5.421203450128514 seconds
time slept for 8.163910723090947 seconds
time slept for 5.110028168514081 seconds
time slept for 9.640176859321892 seconds
time slept for 9.902131201213463 seconds
time slept for 5.5781383243019835 seconds
time slept for 6.295727536016168 seconds
time slept for 5.308405750867421 seconds
time slept for 7.337633821261477 seconds
time slept for 7.008767044838104 seconds
time slept for 7.765438601864929 seconds
time slept for 5.529144476597819 seconds
time slept for 5.

In [ ]:
#tutorial for google search parameter: https://cleverclicks.com.au/blog/15-awesome-google-search-tricks/